In [1]:
# Librerie base
import pandas as pd

import numpy as np

import os

# Modelli
import sklearn as skl
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

import joblib
from sklearn.preprocessing import FunctionTransformer

# Flask
from flask import Flask, render_template, request

In [2]:
# Definizione dei percorsi e dei parametri globali
DATASET_BASE_PATH: str = './datasets'
COMBATS_PATH: str = f"{DATASET_BASE_PATH}/combats.csv"
TYPE_CHART_PATH: str = f"{DATASET_BASE_PATH}/type_chart.csv"
POKEMON_PATH: str = f"{DATASET_BASE_PATH}/pokemon.csv"
MODEL_DIR: str = './models'
RANDOM_STATE: int = 42
# Dimensioni dei set di dati
TEST_SIZE: float = 0.2        # 20% per il test
VALIDATION_SIZE: float = 0.2  # 20% per la validazione
                              # 60% per l'addestramento

In [3]:
def load_pokemon_data(combats_path: str, pokemon_path: str) -> pd.DataFrame:
   df_combats = pd.read_csv(combats_path)
   df_pokemon = pd.read_csv(pokemon_path, index_col="#").fillna("None")
   # Prerprocessing del campo winner per avere 0 se vince il primo Pokemon e 1 se vince il_So
   df_combats["Winner"] = (df_combats["Winner"] != df_combats["First_pokemon"]).astype(int)
   # Merge first and second Pokémon stats
   return df_combats.merge(
      df_pokemon.add_suffix("_F"), how="left",
      left_on="First_pokemon", right_index=True
   ).merge(
      df_pokemon.add_suffix("_S"), how="left",
      left_on="Second_pokemon", right_index=True
   ).drop(
      columns=["First_pokemon", "Second_pokemon"]
   )

In [4]:
# Caricamento dei DataFrame
df_type_chart = pd.read_csv(TYPE_CHART_PATH).fillna("None")
df_pokemon = pd.read_csv(POKEMON_PATH, index_col="#").fillna("None")
df_combats: pd.DataFrame = load_pokemon_data(COMBATS_PATH, POKEMON_PATH)

In [5]:
def function_transformer(dataframe):
   # Funzione per calcolare i moltiplicatori di tipo
   def get_dual_multiplier(df, atk_col, def1_col, def2_col, newcol):
      tmp = df_type_chart.rename(columns={
         'attack': atk_col,
         'defense1': def1_col,
         'defense2': def2_col
      })
      df = df.merge(
         tmp[[atk_col, def1_col, def2_col, 'multiplier']],
         on=[atk_col, def1_col, def2_col],
         how='left'
      ).rename(columns={'multiplier': newcol})
      df[newcol] = df[newcol].fillna(1.0)
      return df
   # Aggiunta dei moltiplicatori di tipo
   dataframe: pd.DataFrame = dataframe.copy()
   dataframe: pd.DataFrame = get_dual_multiplier(dataframe, 'Type 1_F', 'Type 1_S', 'Type 2_S', 'F1_to_S')
   dataframe: pd.DataFrame = get_dual_multiplier(dataframe, 'Type 2_F', 'Type 1_S', 'Type 2_S', 'F2_to_S')
   dataframe: pd.DataFrame = get_dual_multiplier(dataframe, 'Type 1_S', 'Type 1_F', 'Type 2_F', 'S1_to_F')
   dataframe: pd.DataFrame = get_dual_multiplier(dataframe, 'Type 2_S', 'Type 1_F', 'Type 2_F', 'S2_to_F')
   # Rimozione delle colonne di tipo originali
   dataframe.drop(columns=['Type 1_F', 'Type 2_F', 'Type 1_S', 'Type 2_S'], inplace=True)
   # Rimpiazzo delle statistiche con la differenza delle statistiche tra i due pokemon
   stats: list[str] = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
   for stat in stats:
      dataframe[f"delta_{stat}"] = dataframe[f'{stat}_F'] - dataframe[f'{stat}_S']
      dataframe.drop(columns=[f'{stat}_F', f'{stat}_S'], inplace=True)
   # Rimozione delle colonne non necessarie
   return dataframe.drop(columns=['Name_F', 'Name_S', 'Generation_F', 'Generation_S'])

data_transformer: FunctionTransformer = FunctionTransformer(func=function_transformer).set_output(transform="pandas")

In [6]:
app: Flask = Flask(__name__, template_folder='webpage')

models = os.listdir(MODEL_DIR)
model = joblib.load(os.path.join(MODEL_DIR, models[0]))

@app.route('/', methods=['GET', 'POST'])
def one_v_one_battle():
    pokemon_first: str | None = None
    pokemon_second: str | None = None
    winner: str | None = None
    confidence: float | None = None
    if request.method == 'POST':
        if request.form.get('change_model'):
            model_filename: str = request.form.get('model_file')
            if model_filename in models:
                global model
                model = joblib.load(os.path.join(MODEL_DIR, model_filename))
        if request.form.get('predict_fight'):
            pokemon_first = request.form.get('pokemon_first')
            pokemon_second = request.form.get('pokemon_second')
            first_data = df_pokemon.loc[df_pokemon['Name'] == pokemon_first].add_suffix(suffix='_F')
            second_data = df_pokemon.loc[df_pokemon['Name'] == pokemon_second].add_suffix(suffix='_S')
            winner_id = model.predict(first_data.merge(second_data, how='cross'))
            winner = pokemon_first if winner_id == 0 else pokemon_second
            confidence = np.max(model.predict_proba(first_data.merge(second_data, how='cross')), axis=1)[0]* 100
    return render_template('1v1_battle.html', models=models, selected_pokemons=[pokemon_first, pokemon_second], pokemon_choices=df_pokemon['Name'], winner=winner, confidence=confidence)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Jun/2025 16:00:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:00:25] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:00:28] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:00:31] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:00:33] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:00:33] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:00:33] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:00:38] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:00:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:00:59] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:02:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:02:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:02:51] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:02:54] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:02:55] "POST / HTTP/1.1" 200 -
